In [5]:
from datetime import datetime
import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import html
from markdownify import markdownify 

endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"

query_txt = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT ?textItem ?kuratorLabel ?textUrl
WHERE
{
  <placeholder>
  ?textItem cpsp:P46 ?kuratorStatement. 
  ?kuratorStatement cpsps:P46 ?kuratorItem. 
  ?kuratorItem rdfs:label ?kuratorLabel.
  ?textItem cpsp:P57 ?urlstatement. 
  ?urlstatement cpsps:P57 ?textUrl. 
}"""

query_img = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT DISTINCT ?itemLabel ?itemDescr ?imgItem ?imgUrl (YEAR(?publishDate) as ?publishDate)
WHERE
{
  ?imgItem cpsp:P107 ?urlStatement. 
  ?urlStatement cpsps:P107 ?imgUrl. 
  ?imgItem cpsp:P60 ?dateStatement. 
  ?dateStatement cpsps:P60 ?publishDate. 
  ?imgItem cpsp:P6 ?partOfStatement.
  ?partOfStatement cpsps:P6 ?partOfItem.
  <placeholder> 

  SERVICE wikibase:label {
      bd:serviceParam wikibase:language "en,de".
      ?imgItem rdfs:label ?itemLabel.
      ?imgItem schema:description ?itemDescr.
    }
}"""

query_graph = """PREFIX cps: <https://computational-publishing-service.wikibase.cloud/entity/>
PREFIX cpss: <https://computational-publishing-service.wikibase.cloud/entity/statement/>
PREFIX cpsv: <https://computational-publishing-service.wikibase.cloud/value/>
PREFIX cpspt: <https://computational-publishing-service.wikibase.cloud/prop/direct/>
PREFIX cpsp: <https://computational-publishing-service.wikibase.cloud/prop/>
PREFIX cpsps: <https://computational-publishing-service.wikibase.cloud/prop/statement/>
PREFIX cpspq: <https://computational-publishing-service.wikibase.cloud/prop/qualifier/>

SELECT ?x ?y
WHERE
{
  ?a cpsp:P2 ?c.
  ?c cpsps:P2 ?d.
  ?a rdfs:label ?x.
  ?d rdfs:label ?y.

}LIMIT 1"""

query_graph2 = """
SELECT ?a ?b ?c
WHERE
{
  ?a rdfs:label ?c
}LIMIT 100"""


def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_text(textitem_id):
    q = ""
    if textitem_id:
        q = query_txt.replace("<placeholder>", "cps:"+textitem_id+" cpsp:P46 ?kuratorStatement.")
    else:
        q = query_txt.replace("<placeholder>","")

    results_txt = run_query(endpoint_url, q)
    for item in results_txt["results"]["bindings"]:
        print('Wikibase link: ' + '[' + item['textItem']['value'] + ']' + '(' + item['textItem']['value'] + ')' + '\n')
        print('Kurator: ' + item['kuratorLabel']['value'] + '\n')
        headers = {'User-Agent': 'Ex_Books_conference_bot/0.0 (https://github.com/SimonXIX/Experimental_Books_workshop; ad7588@coventry.ac.uk)'}
        r = requests.get(item['textUrl']['value'], headers=headers, stream=True)
        text = str(r.text) #changed from r.content
        text = text.removeprefix("<!DOCTYPE html>") #changed from "b'<!DOCTYPE html>"
        print(markdownify(text)) #changed

def get_delay(date):
    try:
        date = datetime.datetime.strptime(date, '%a, %d %b %Y %H:%M:%S GMT')
        timeout = int((date - datetime.datetime.now()).total_seconds())
    except ValueError:
        timeout = int(date)
    return timeout

def get_img(partOfItem_id):

    q = ""
    if partOfItem_id:
        q = query_img.replace("<placeholder>", "?partOfStatement cpsps:P6 cps:"+partOfItem_id+".")
    else:
        q = query_img.replace("<placeholder>","")

    results_img = run_query(endpoint_url, q)

    for item in results_img["results"]["bindings"]:    

        title = item['itemLabel']['value']
        description = html.unescape(item['itemDescr']['value'])

        print('\nWikibase link: ' + '[' + item['imgItem']['value'] + ']' + '(' + item['imgItem']['value'] + ')' + '\n')
        print('Title: ' + title + '\n')
        print('Year: ' + item['publishDate']['value'] + '\n')
        print('Description: ' + description + '\n')

        # get image from image URL
        image_url=item['imgUrl']['value']

        #display image with title + alt text (in markdown)
        print('!['+ title +']('+image_url+'){fig-alt="'+description+'"}\n\n')

def get_graph():
    import VizKG.visualize as vkg
    results_graph1 = run_query(endpoint_url, query_graph)
    #print(results_graph1)
    #print('---')
    results_graph2 = run_query(endpoint_url, query_graph2)
    #print(results_graph2)

    chart = vkg(sparql_query=query_graph2, sparql_service_url=endpoint_url, chart='wordcloud')
    chart.plot()

In [6]:
get_text("Q252")
get_text("Q253")
get_text("Q254")
get_text("Q255")
get_text("Q256")
get_text("Q257")
get_text("Q258")
get_text("Q259")
get_text("Q260")
get_text("Q262")
get_text("Q263")

Wikibase link: [https://computational-publishing-service.wikibase.cloud/entity/Q252](https://computational-publishing-service.wikibase.cloud/entity/Q252)

Kurator: Seeger, Ulrike









### Belagerung I: „Vestung Tottis, wie die von den Christen bei der Nacht erobert worden, 1590“



 Breites Format. Vorne rechts ins Bild hineinreitende Reiter mit großen Fahnen. Im Hintergrund die ungarische Festung Totis (Tata) nach dem Vorbild von Sibmachers Kupferstich, der allerdings eine Eroberung durch die Christen aus dem Jahr 1597 wiedergibt. Die sehr dunkle Szenerie wird von zwei Laternen spärlich erleuchtet. Da Totis nicht 1590, sondern 1597 und 1598 durch die Christen erobert wurde, und zudem zu den zeitlich als nächste dargestellten Belagerungen eine Zeitspanne von vier Jahren liegt, kann es gut sein, dass der Jahreszahl 1590 ein Versehen zugrunde liegt.
 





Wikibase link: [https://computational-publishing-service.wikibase.cloud/entity/Q253](https://computational-publishing-service.wik

<hr>

In [7]:
get_img("Q208")


Wikibase link: [https://computational-publishing-service.wikibase.cloud/entity/Q283](https://computational-publishing-service.wikibase.cloud/entity/Q283)

Title: Die barocken Schloss- und Gartenveduten bild

Year: 2018

Description: Bild für Die barocken Schloss- und Gartenveduten

![Die barocken Schloss- und Gartenveduten bild](https://previous.bildindex.de/bilder/fmd10005861a.jpg){fig-alt="Bild für Die barocken Schloss- und Gartenveduten"}



Wikibase link: [https://computational-publishing-service.wikibase.cloud/entity/Q288](https://computational-publishing-service.wikibase.cloud/entity/Q288)

Title: Die barocken Schloss- und Gartenveduten bild 2

Year: 2018

Description: Zweites Bild für Die barocken Schloss- und Gartenveduten

![Die barocken Schloss- und Gartenveduten bild 2](https://previous.bildindex.de/bilder/fmd10005863a.jpg){fig-alt="Zweites Bild für Die barocken Schloss- und Gartenveduten"}




<hr>

In [8]:
#get_graph()